# 50 - Ollama Haystack integration demo
https://github.com/deepset-ai/haystack-integrations/blob/main/integrations/ollama.

**WORKING**: haystack env


In [1]:
from haystack import Document, Pipeline
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.document_stores.in_memory import InMemoryDocumentStore

In [2]:
from haystack_integrations.components.embedders.ollama.document_embedder import OllamaDocumentEmbedder
from haystack_integrations.components.embedders.ollama.text_embedder import OllamaTextEmbedder

In [3]:

from haystack import Document, Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore

# from haystack_integrations.components.generators.ollama import OllamaGenerator

In [4]:
document_store = InMemoryDocumentStore()
document_store.write_documents(
    [
        Document(content="Super Mario was an important politician"),
        Document(content="Mario owns several castles and uses them to conduct important political business"),
        Document(
            content="Super Mario was a successful military leader who fought off several invasion attempts by "
            "his arch rival - Bowser"
        ),
    ]
)

3

In [5]:
template = """
Given only the following information, answer the question.
Ignore your own knowledge.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ query }}?
"""

In [6]:
from haystack_integrations.components.generators.ollama import OllamaGenerator

In [7]:
pipe = Pipeline()

pipe.add_component("retriever", InMemoryBM25Retriever(document_store=document_store))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", OllamaGenerator(model="orca-mini"))
# pipe.add_component("llm", OllamaGenerator(model="zephyr"))
# pipe.add_component("llm", OllamaGenerator(model="orca-mini", url="http://dell:11434/api/generate"))
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [8]:
query = "Who is Super Mario?"
response = pipe.run({"prompt_builder": {"query": query}, "retriever": {"query": query}})

Ranking by BM25...:   0%|          | 0/3 [00:00<?, ? docs/s]

In [9]:
print(response["llm"]["replies"])

[' Based on the given context, Super Mario is likely to be a character from the Mario franchise.']
